In [1]:
# !pip install pyarrow
# !pip install fastparquet 

# Data Loading and columns setting

In [3]:
import warnings
warnings.filterwarnings('ignore')
import sklearn

import matplotlib.pyplot as plt

import seaborn as sns

import numpy as np
import pandas as pd

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.float_format = '{:.3f}'.format


In [5]:
total_df = pd.read_parquet('data/total_df.parquet', engine = 'pyarrow') # fastparquet

total_df = total_df[total_df['year'] != 2021] # we will not be using 2021 because the data is flawed and incomplete
print(total_df.shape)

(8406174, 38)


In [7]:
# setting numerical columns and categorical columns

# these are the numerical columns that we will actually use 
selected_numerical_columns = ['m (kg)',  'At1 (mm)', 'ec (cm3)']

# we are not using 'year' as a feature. the reason for that is when we later predict future 
# Co2 emission of a car, the car's data can be from 2023, for example, and the 2023 isn't in the 
# feature category


# these are the categorical columns that we will actually use 
selected_categorical_columns = ['Ct', 'Mp', 'Ft', 'Fm', 'Country'] 
# Mh has about 200 values, and it only increases 1%. so we will not use it 
### We are not using Cn, model name

all_year = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020] # we will not be using 2021

# Supervised Learning

## Preprocessing the data

In [9]:
target_year = all_year # 2010~2020
target_df = total_df[total_df['year'].isin(target_year)]
print(target_df.shape)

label_column = ['Enedc (g/km)']
feature_columns = selected_numerical_columns + selected_categorical_columns
target_columns = feature_columns + label_column

target_df = target_df[target_columns]
target_df.reset_index(drop=True, inplace=True)
print(target_df.shape)

target_df.loc[target_df['Ct'] == '', 'Ct'] = 'empty' # in order to prevent ohe error 
# 나중에 OHE 할때, value 값이 컬럼명이 되는데, 그때 컬럼명이 빈칸이 되어서 오류가 난다. 그것을 방지
########### None 값은 오류 안냄? 나중에 확인 필요

# let's drop rows that any value is nan
target_df.dropna(subset=target_df.columns.difference(label_column)).shape
print(target_df.shape)

# delete duplicate rows
target_df = target_df.dropna().drop_duplicates()
target_df.reset_index(drop=True, inplace=True)
print('shape of the non-duplicate data from 2010~2020', target_df.shape)

(8405437, 38)
(8405437, 9)
(8405437, 9)
shape of the non-duplicate data from 2010~2020 (862446, 9)


In [88]:
target_df.head()

,m (kg),At1 (mm),ec (cm3),Ct,Mp,Ft,Fm,Country,Enedc (g/km)
0,1340.000,1484.000,1995.000,m1,bmw,petrol,m,gb,143.000
1,1395.000,1484.000,1995.000,m1,bmw,diesel,m,gb,119.000
2,1350.000,1484.000,1995.000,m1,bmw,petrol,m,gb,143.000
3,1495.000,1474.000,1995.000,m1,bmw,diesel,m,gb,135.000
4,1360.000,1484.000,1995.000,m1,bmw,petrol,m,gb,154.000


## Data Split

In [16]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(target_df[feature_columns], target_df[label_column], 
                                                      test_size=0.2,
                                                      shuffle=True)

x_train.reset_index(drop=True, inplace=True)
x_valid.reset_index(drop=True, inplace=True)

## Scaling

In [17]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler = MinMaxScaler()
x_train[selected_numerical_columns] = scaler.fit_transform(x_train[selected_numerical_columns])
x_valid[selected_numerical_columns] = scaler.transform(x_valid[selected_numerical_columns])

In [18]:
for column in selected_categorical_columns:
    if not set(x_valid[column].unique()).issubset(set(x_train[column].unique())):
        print(x_train[column].unique())
        print(x_valid[column].unique())

### There should be no result from this cell ####
### if there is an output, go back to data split and re-run

## One Hot Encoding

In [19]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse = False) 

ohe.fit(x_train[selected_categorical_columns])

new_x_train_cat = ohe.transform(x_train[selected_categorical_columns])
new_x_valid_cat = ohe.transform(x_valid[selected_categorical_columns])

# ohe columns
ohe_columns = []
for idx in range(len(ohe.categories_)):
    ohe_columns += ohe.categories_[idx].tolist()

# pd.Dataframe
new_x_train_cat = pd.DataFrame(new_x_train_cat, columns= ohe_columns)
new_x_valid_cat = pd.DataFrame(new_x_valid_cat, columns= ohe_columns)


In [20]:
# the number of rows matches
print(x_train.shape, new_x_train_cat.shape)
print(x_valid.shape, new_x_valid_cat.shape)

# delete original categorical columns
x_train.drop(columns=selected_categorical_columns, inplace=True)
x_valid.drop(columns=selected_categorical_columns, inplace=True)

# add OHE
x_train = pd.concat([x_train, new_x_train_cat], axis=1)
x_valid = pd.concat([x_valid, new_x_valid_cat], axis=1)


print(x_train.shape)
print(x_valid.shape)

(689956, 8) (689956, 101)
(172490, 8) (172490, 101)
(689956, 104)
(172490, 104)


In [89]:
x_train.head()

,m (kg),At1 (mm),ec (cm3),empty,m1,m1g,n1,n1g,n2,bmw,bmw group,daimler,daimler ag,fca,fca italy spa,fca-tesla,fiat group automobiles spa,ford pool,ford-volvo,ford-werke gmbh,general motors,gm,honda,honda motor europe ltd,hyundai,jlt pool,kia,mercedes-benz,mg-saic,mitsubishi motors,mitsubishi pool,na,pool renault,psa-opel,renault,renault-nissan-mitsubishi,suzuki,suzuki pool,tata motors jaguar land rover,"tata motors ltd, jaguar cars ltd , land rover","tata motors ltd, jaguar cars ltd, land rover",tjl,toyota -daihatsu group,toyota-dahaitsu group,toyota-daihatsu group,toyota-mazda,vw group pc,vw-saic,biodiesel,cng,diesel,diesel-electric,e85,electric,hybrid-petrol-e,hydrogen,lpg,ng,ng-biomethane,ng_biomethane,other,petrol,petrol phev,petrol-electric,petrol-gas,2,b,e,f,h,m,n,na,p,at,be,bg,cy,cz,de,dk,ee,es,fi,fr,gb,gr,hr,hu,ie,is,it,lt,lu,lv,mt,nl,no,pl,pt,ro,se,si,sk
0,0.212,0.094,0.188,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,0.218,0.091,0.234,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,0.354,0.099,0.252,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,0.267,0.091,0.588,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,0.213,0.089,0.235,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,

## Modeling

In [21]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from statsmodels.stats.outliers_influence import variance_inflation_factor

### Linear Regression (without cross validation)

When we conduct a single linear regression, the result is as follows. 

In [46]:
lr = LinearRegression()

lr.fit(x_train, y_train)

y_pred = lr.predict(x_valid)

mse = mean_squared_error(y_valid , y_pred)
rmse = np.sqrt(mse)

print('Linear Regression, R2 : {:.4f}'.format(r2_score(y_valid, y_pred)))
print('Linear Regression, RMSE : {:.4f}'.format(rmse))


Linear Regression, R2 : 0.7712
Linear Regression, RMSE : 18.5429


#### Checking multi colinearity

##### 1. VIF method from statsmodel

In [ ]:
num_data = target_df[selected_numerical_columns]

vif_data = pd.DataFrame()
vif_data["feature"] = num_data.columns
vif_data["VIF"] = [variance_inflation_factor(num_data.values, i) for i in range(len(num_data.columns))]
vif_data.sort_values(by='VIF', ascending=False)

,feature,VIF
0,m (kg),41.0967
1,At1 (mm),23.7489
2,ec (cm3),14.6800


VIF values are acceptable

##### 2. checking correlations between columns

In [ ]:
target_df[selected_numerical_columns].corr()

,m (kg),At1 (mm),ec (cm3)
m (kg),1.0000,0.2631,0.6578
At1 (mm),0.2631,1.0000,0.1521
ec (cm3),0.6578,0.1521,1.0000


ec and m columns have high corr but for the sake of analysis, we keep the columns

### Linear Regression 

However, when we do cross validation with normal linear regression, the results become very unstable.

In [47]:
lr = LinearRegression()

scores = cross_val_score(lr, x_train, y_train, cv=5, scoring='r2')
scores

array([-2.31953211e+14,  7.72501211e-01,  7.69659269e-01, -1.98744279e+17,
       -4.20704944e+17])

This overfitting can be partially resolved with regularization methods.

### Lasso Linear Regression

The problem is somewhat solved with Lasso Regression

In [43]:
lasso_lr = Lasso(random_state=42)

scores = cross_val_score(lasso_lr, x_train, y_train, cv=5, scoring='r2')
scores

array([0.43680532, 0.44099168, 0.43720041, 0.4392151 , 0.43532796])

x_valid, y_valid are used to make the final prediction after finding the best hyper parameters with GridSearchCV.

But at this stage, we don't do hyperparameter tuning just yet

### Ridge Linear Regression

Ridge regression greatly improves the r2 score

In [44]:
ridge_lr = Ridge(random_state=42)

scores = cross_val_score(ridge_lr, x_train, y_train, cv=5, scoring='r2')
scores

array([0.76452343, 0.77250773, 0.76965904, 0.7718569 , 0.76734422])

x_valid, y_valid are used to make the final prediction after finding the best hyper parameters with GridSearchCV.

But at this stage, we don't do hyperparameter tuning just yet

### RandomForest Regressor

Randomforest regressor greatly improves r2 score, and other metrics as well. 

In [42]:
### don't run this cell !!! it takes more than 30 min

rf_regr = RandomForestRegressor(n_jobs = -1)

scores = cross_val_score(rf_regr, x_train, y_train, cv=5, scoring='r2')
scores

array([0.93892922, 0.94193243, 0.94143147, 0.94222469, 0.93924838])

Eventhough randomforest regressor performs the best, due to the limit of computing power, we will use MLP regressor to tune the hyper parameters and expand the analysis

Below, we make a function to create a dataframe that summarizes the results of metrics and models

In [78]:
metrics =['r2', 'neg_mean_squared_error', 'neg_root_mean_squared_error', 'neg_mean_absolute_error']

def create_cross_val(model, metrics):
    
    mean_list = []
    std_list = []

    for metric in metrics:
        scores = cross_val_score(model, x_train, y_train, cv=5, scoring=metric)

        if scores[0] < 0:
            scores = [x * (-1) for x in scores]

        mean_list.append(np.mean(scores))
        std_list.append(np.std(scores))
    
    return mean_list, std_list

In [81]:
mean_result_df = pd.DataFrame()
std_result_df = pd.DataFrame()

mean_result_df['Metrics'] = ['R2', 'MSE', 'RMSE', 'MAE']
std_result_df['Metrics'] = ['R2', 'MSE', 'RMSE', 'MAE']

In [82]:
# I created a fuction for each model separately because randomforest took too much time, and kernel kept dying.

lasso_lr = Lasso(random_state=42)
mean_list, std_list = create_cross_val(lasso_lr, metrics)
mean_result_df['Lasso'] = mean_list
std_result_df['Lasso'] = std_list


ridge_lr = Ridge(random_state=42)
mean_list, std_list = create_cross_val(ridge_lr, metrics)
mean_result_df['Ridge'] = mean_list
std_result_df['Ridge'] = std_list

In [85]:
rf_regr = RandomForestRegressor(random_state=42, n_jobs=-1)
mean_list, std_list = create_cross_val(rf_regr, metrics)
mean_result_df['RF'] = mean_list
std_result_df['RF'] = std_list

This is the mean scores of 3 different models on each metric

In [86]:
mean_result_df

,Metrics,Lasso,Ridge,RF
0,R2,0.438,0.769,0.941
1,MSE,841.525,345.571,88.553
2,RMSE,29.009,18.589,9.410
3,MAE,21.432,13.074,5.046


This is the std of scores of 3 different models on each metric

In [87]:
std_result_df

,Metrics,Lasso,Ridge,RF
0,R2,0.002,0.003,0.001
1,MSE,7.281,5.235,2.257
2,RMSE,0.125,0.141,0.120
3,MAE,0.060,0.037,0.014
